# Importing Libraries

In [ ]:
!pip install transformers torch datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
from datasets import load_dataset,DatasetDict

In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["document"],
        max_length=4096,  # Longformer max length
        truncation=True,
        padding="max_length"
    )

In [ ]:
from transformers import LongformerTokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")


def tokenize_function(examples):
    return tokenizer(
        examples["document"],
        max_length=4096,  # Set max length for Longformer
        truncation=True,
        padding="max_length"
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from datasets import load_dataset


dataset = load_dataset("EdinburghNLP/xsum")

new_dataset = DatasetDict({
    "train": dataset['train'].shuffle(seed = 42).select(range(30000)),
    "validation": dataset['validation'].shuffle(seed = 43).select(range(2000)),
    "test":dataset['test'].shuffle(seed = 43).select(range(2000)),
})

tokenized_dataset = new_dataset.map(tokenize_function, batched=True, remove_columns=["document", "summary"])


In [ ]:
def preprocess_function(data):
    inputs = tokenizer(
        data["document"],
        max_length=4096,
        truncation=True,
        padding="max_length"
    )
    targets = tokenizer(
        data["summary"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    inputs["labels"] = targets["input_ids"]
    return inputs

In [ ]:
from transformers import LEDTokenizer, LEDForConditionalGeneration, Trainer, TrainingArguments

tokenizer = LEDTokenizer.from_pretrained("allenai/led-base-16384")
model = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384")

In [ ]:
tokenized_train_dataset = new_dataset['train'].map(preprocess_function, batched=True)
tokenized_validation_dataset = new_dataset['validation'].map(preprocess_function, batched=True)

In [ ]:
from transformers import TrainingArguments,Trainer

training_args = TrainingArguments(
    output_dir="./LED_xsum_pretrained_1.0",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
    evaluation_strategy="steps",
    eval_steps=10_000,
    fp16=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    tokenizer=tokenizer,
)

<ipython-input-9-ab76ad38ada4>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: swanididde (swanididde-university-of-colorado-boulder). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
10000,0.522900,0.500458
20000,0.499500,0.463755
30000,0.451100,0.438460


TrainOutput(global_step=30000, training_loss=0.5193329015096029, metrics={'train_runtime': 5705.0746, 'train_samples_per_second': 5.258, 'train_steps_per_second': 5.258, 'total_flos': 8.100614504448e+16, 'train_loss': 0.5193329015096029, 'epoch': 1.0})

In [ ]:
model.save_pretrained("./LED_xsum_pretrained1")
tokenizer.save_pretrained("./LED_xsum_pretrained1")

('./LED_xsum_pretrained1/tokenizer_config.json',
 './LED_xsum_pretrained1/special_tokens_map.json',
 './LED_xsum_pretrained1/vocab.json',
 './LED_xsum_pretrained1/merges.txt',
 './LED_xsum_pretrained1/added_tokens.json')

In [ ]:
import shutil
import os
os.makedirs("./models", exist_ok=True)
shutil.make_archive("LED_model", 'zip', "./LED_xsum_pretrained1")

'/content/LED_model.zip'

In [ ]:
import transformers
print(transformers.__version__)

4.46.3


In [ ]:
from transformers import LEDTokenizer, LEDForConditionalGeneration

model = LEDForConditionalGeneration.from_pretrained("./LED_xsum_pretrained1")
tokenizer = LEDTokenizer.from_pretrained("./LED_xsum_pretrained1")


In [ ]:
def preprocess_function(examples):
    inputs = tokenizer(
        examples["document"],
        max_length=1024,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    labels = tokenizer(
        examples["summary"],
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    inputs["labels"] = labels["input_ids"]
    return inputs

In [ ]:
tokenized_validation = new_dataset['validation'].select(range(10)).map(preprocess_function, batched=True)

In [ ]:
predictions = []
references = []
model.eval()
for example in tokenized_validation:
    inputs = tokenizer(
        example["document"], return_tensors="pt", max_length=4096, truncation=True
    )
    outputs = model.generate(inputs["input_ids"], max_length=128, num_beams=4)
    predicted_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    predictions.append(predicted_summary)
    references.append(example["summary"])  # Replace with your reference column


In [ ]:
predictions[0]

'The leader of the Welsh Liberal Democrats has said the party has " punched above its weight" in the general election.'

In [ ]:
!pip install rouge_score
from rouge_score import rouge_scorer

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
results = rouge.compute(predictions=predictions, references=references)
print(results)

{'rouge1': 0.30611106099075425, 'rouge2': 0.12974837662337663, 'rougeL': 0.233816520954271, 'rougeLsum': 0.2324661373225574}
